# `Worklist` Basics
The most flexible way to give pipetting instructions to the Tecan EVOware are worklist files (`*.gwl`).

A simple worklist might look something like this:
```
W1;
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;1;;50.00;;;;
W1;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;2;;50.00;;;;
```

Between the `;`, the EVOware accepts various parameters such as the _liquid class_ or _rack type_.
As you might guess from the above example you don't want to write such worklist files by hand.

Instead, `robotools` can write these worklists for you.
This is done with the `Worklist` class, a [context manager](https://book.pythontips.com/en/latest/context_managers.html) that automatically creates worklists from high-level liquid handling operations.

In [1]:
import robotools

## Labware Definition

In [2]:
# Let's say we have 6 1.5 mL Eppis with 500 µL sample each.
samples = robotools.Labware('Samples', 6, 1, min_volume=20, max_volume=1500, initial_volumes=500)

# We want to run an assay in a microtiterplate
assay = robotools.Labware('AssayMTP', 8, 12, min_volume=30, max_volume=300)

# We have a trough with assay mastermix
mastermix = robotools.Trough('MasterMix', 8, 1, min_volume=5000, max_volume=25_000, initial_volumes=10_000)

print(samples)
print(assay)
print(mastermix)

Samples
[[500.]
 [500.]
 [500.]
 [500.]
 [500.]
 [500.]]
AssayMTP
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
MasterMix
[[10000.]]


## Creating the `Worklist`
The `Worklist` has a ton of methods. It doesn't make sense to list them all here, but you can use `help(robotools.Worklist)` or look them up in the documentation.

The most important method is `Worklist.transfer`:

In [3]:
help(robotools.Worklist.transfer)

Help on function transfer in module robotools.evotools:

transfer(self, source: robotools.liquidhandling.Labware, source_wells: Union[str, Sequence[str], numpy.ndarray], destination: robotools.liquidhandling.Labware, destination_wells: Union[str, Sequence[str], numpy.ndarray], volumes: Union[float, Sequence[float], numpy.ndarray], *, label: Union[str, NoneType] = None, wash_scheme: int = 1, partition_by: str = 'auto', **kwargs)
    Transfer operation between two labwares.
    
    Parameters
    ----------
    source : liquidhandling.Labware
        Source labware
    source_wells : str or iterable
        List of source well ids
    destination : liquidhandling.Labware
        Destination labware
    destination_wells : str or iterable
        List of destination well ids
    volumes : float or iterable
        Volume(s) to transfer
    label : str
        Label of the operation to log into labware history
    wash_scheme : int
        Wash scheme to apply after every tip use
    part

In [4]:
n_samples = samples.n_rows
n_replicates = 3

with robotools.Worklist("02_assay.gwl") as wl:
    wl.wash()
    # Add triplicates of the samples
    for r in range(n_replicates):
        wl.transfer(
            samples, samples.wells,
            assay, assay.wells[:n_samples, r],
            10,
            label=f'Add replicate {r+1} samples'
        )
    
    # Select trough wells to aspirate from
    trough_wells = robotools.get_trough_wells(
        n=n_samples*n_replicates,
        trough_wells=mastermix.wells[:,0]
    )
    
    # Add buffer to all wells
    wl.transfer(
        mastermix, trough_wells,
        assay, assay.wells[:n_samples,:n_replicates],
        50,
        label='Add mastermix'
    )

Our three labwares were modified by the pipetting steps. We can inspect the history of the 96-well plate to check if we selected the right wells & volumes:

In [5]:
print(assay.report)

AssayMTP
initial
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Add replicate 1 samples
[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

Add replicate 2 samples
[[10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 1

If a `filepath` is passed, the `Worklist` automatically writes the worklist (`wl`) to it, overwriting any previous content.

```python
with Worklist(filepath=...) as wl:
    ...
```

Note that it includes the `label`s as comments.

In [6]:
with open("02_assay.gwl") as wlfile:
    print(wlfile.read())

W1;
C;Add replicate 1 samples
A;Samples;;;1;;10.00;;;;
D;AssayMTP;;;1;;10.00;;;;
W1;
A;Samples;;;2;;10.00;;;;
D;AssayMTP;;;2;;10.00;;;;
W1;
A;Samples;;;3;;10.00;;;;
D;AssayMTP;;;3;;10.00;;;;
W1;
A;Samples;;;4;;10.00;;;;
D;AssayMTP;;;4;;10.00;;;;
W1;
A;Samples;;;5;;10.00;;;;
D;AssayMTP;;;5;;10.00;;;;
W1;
A;Samples;;;6;;10.00;;;;
D;AssayMTP;;;6;;10.00;;;;
W1;
C;Add replicate 2 samples
A;Samples;;;1;;10.00;;;;
D;AssayMTP;;;9;;10.00;;;;
W1;
A;Samples;;;2;;10.00;;;;
D;AssayMTP;;;10;;10.00;;;;
W1;
A;Samples;;;3;;10.00;;;;
D;AssayMTP;;;11;;10.00;;;;
W1;
A;Samples;;;4;;10.00;;;;
D;AssayMTP;;;12;;10.00;;;;
W1;
A;Samples;;;5;;10.00;;;;
D;AssayMTP;;;13;;10.00;;;;
W1;
A;Samples;;;6;;10.00;;;;
D;AssayMTP;;;14;;10.00;;;;
W1;
C;Add replicate 3 samples
A;Samples;;;1;;10.00;;;;
D;AssayMTP;;;17;;10.00;;;;
W1;
A;Samples;;;2;;10.00;;;;
D;AssayMTP;;;18;;10.00;;;;
W1;
A;Samples;;;3;;10.00;;;;
D;AssayMTP;;;19;;10.00;;;;
W1;
A;Samples;;;4;;10.00;;;;
D;AssayMTP;;;20;;10.00;;;;
W1;
A;Samples;;;5;;10.00;;;;
D;As